Install LIME

In [ ]:
pip install lime

Import packages

In [ ]:
# Import necessary libraries

import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import make_pipeline
from lime.lime_text import LimeTextExplainer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import OrderedDict

Load training data

In [ ]:
# Load training data
train_df = pd.read_csv("/content/Data/train.csv")
print("Train shape : ", train_df.shape)
print(train_df.head)

Display rows with NaN values

In [ ]:
# Display rows with NaN values
nan_rows = train_df[train_df.isna().any(axis=1)]
print(nan_rows)

Print train

In [ ]:
# Remove rows with NaN values
train_df = train_df.dropna()
print("Train shape : ", train_df.shape)
print(train_df.head)

Train and val data

In [ ]:
# Split the data into training and validation sets
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=2018)

In [ ]:
print(val_df)

Select sample val data

In [ ]:
# Select specific rows from validation set based on qid for inspection
df_select = pd.concat([val_df[val_df['qid'] == 'd61b098340966d9d6501'], val_df[val_df['qid'] == 'feb0053f32eda8483c9f']], axis=0)

Select question

In [ ]:
# Display the 'question_text' column of selected rows
df_select.question_text

Print val data head

In [ ]:
# Reset index of validation dataframe
val_df.reset_index(drop=True, inplace=True)
print(val_df)

TF-IDF vectorizer

In [ ]:
# Create a TF-IDF vectorizer and transform the training and validation data

## vectorize to tf-idf vectors
tfidf_vc = TfidfVectorizer(min_df = 10, max_features = 100000, analyzer = "word", ngram_range = (1, 2), stop_words = 'english', lowercase = True)
train_vc = tfidf_vc.fit_transform(train_df["question_text"])
val_vc = tfidf_vc.transform(val_df["question_text"])

Logistic regression model

In [ ]:
# Train a Logistic Regression model on the training data

model = LogisticRegression(C = 0.5, solver = "sag")
model = model.fit(train_vc, train_df.target)

# Predict on the validation data
val_pred = model.predict(val_vc)

Evaluation metrics

In [ ]:
# Calculate evaluation metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

accuracy = accuracy_score(val_df.target, val_pred)
precision = precision_score(val_df.target, val_pred)
recall = recall_score(val_df.target, val_pred)
f1 = f1_score(val_df.target, val_pred)

In [ ]:
# Print evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

In [ ]:
# Display confusion matrix
conf_matrix = confusion_matrix(val_df.target, val_pred)
print("Confusion Matrix:\n", conf_matrix)

In [ ]:
# Display classification report
# Define class names
class_names = ["sincere", "insincere"]
class_report = classification_report(val_df.target, val_pred, target_names=class_names)
print("Classification Report:\n", class_report)

In [ ]:
# Filter the rows where target is 1
target_1_rows = val_df[val_df['target'] == 1]

# Print the filtered rows and their row indices
print("Rows with target = 1:")
print(target_1_rows)

print("\nRow indices of rows with target = 1:")
print(target_1_rows.index.tolist())

In [ ]:
# Select a specific instance from the validation set for explanation
import numpy as np
prediction_index = 138
idx = int(val_df.index[prediction_index])
# print(idx)
c = make_pipeline(tfidf_vc, model)
class_names = ["sincere", "insincere"]

# Create a LIME text explainer
explainer = LimeTextExplainer(class_names = class_names)

# Explain the prediction for the selected instance
exp = explainer.explain_instance(val_df["question_text"][idx], c.predict_proba, num_features = 10)

# Print the selected question text and its prediction probabilities
print(val_df["question_text"][idx])
print("Probability (Insincere) =", c.predict_proba([val_df["question_text"][idx]])[0, 1])
print("Probability (Sincere) =", c.predict_proba([val_df["question_text"][idx]])[0, 0])
print("True Class is:", class_names[int(val_df["target"][idx])])

In [ ]:
# Get explanation weights as a list of tuples
exp.as_list()

In [ ]:
# Print original prediction probability
print('Original prediction:',  model.predict_proba(val_vc[prediction_index])[0, 1])

# Create a copy of the selected instance's TF-IDF vector and modify specific features
tmp = val_vc[prediction_index].copy()
tmp[0, tfidf_vc.vocabulary_['indians']] = 0
tmp[0, tfidf_vc.vocabulary_['europeans']] = 0

# Print prediction after removing specific features
print('Prediction after removing some features:', model.predict_proba(tmp)[0, 1])

# Print the difference in prediction probabilities
print('Difference:', model.predict_proba(tmp)[0, 1] - model.predict_proba(val_vc[prediction_index])[0, 1])

In [ ]:
# Display LIME explanation in a notebook
exp.show_in_notebook(text=val_df["question_text"][idx], labels=(1,))

In [ ]:
# Extract and plot LIME weights
weights = OrderedDict(exp.as_list())
lime_weights = pd.DataFrame({"words": list(weights.keys()), "weights": list(weights.values())})

# Plot the feature weights
sns.barplot(x = "words", y = "weights", data = lime_weights, palette="viridis")
plt.xticks(rotation = 45)
plt.title("Sample {} features weights given by LIME".format(idx))
plt.show()